## From EXR to YUV
* .exr images from ftp://svtopencontent.svt.se
* 1500 images at 50 fps = 30sec
* inside a .yuv raw video with 4k resolution and yuv420p sampling

In [2]:
import subprocess
c_seq = "ffmpeg -y -start_number 180000 -framerate 50 -i test_vids/src_waterfall/waterfall_%04d.exr \
        -c:v rawvideo -pix_fmt yuv420p -s 3840x2160 -r 50 test_vids/refs/waterfall.yuv"
subprocess.call(c_seq, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

0

## From YUV to YUV multires
the .yuv reference uncompressed file is downsized to 5 more resolutions

In [ ]:
import numpy as np
import subprocess
source = "smithy"
res_widths = [3840, 2560, 1920, 1280, 1024, 640]
res_heights = [2160, 1440, 1080, 720, 576, 360]
for i in range(1, res_widths.size):
    path = "test_vids/downsampled/" + source + "_" + str(res_widths[i-1]) + ".yuv"
    out = "test_vids/downsampled/" + source + "_" + str(res_widths[i]) + ".yuv"
    command = f"ffmpeg -f rawvideo -video_size {res_widths[i-1]}x{res_heights[i-1]} -r 50 -pixel_format yuv420p \
    -i {path} -f rawvideo -c:v rawvideo -filter:v scale={res_widths[i]}x{res_heights[i]} -pix_fmt yuv420p {out}"
    subprocess.call(command, shell=True)

## From YUV to H264 AVC

In [4]:
import os
import subprocess
source = "waterfall"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
#quality = [10, 17, 23, 29, 35]
quality = [10, 14, 17, 20, 23, 26, 29, 32, 35, 40]
for current_crf in range(5,10):
    out = "test_vids/encoded/" + source + "/" + source + "AVC_cf" + str(quality[current_crf]) + ".mp4"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libx264 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps= 12 q=-1.0 Lsize=   86232kB time=00:00:29.94 bitrate=23594.2kbits/s speed=0.241x     
video:86213kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.022410%
[libx264 @ 0x5629c9437180] frame I:6     Avg QP:34.00  size:359283
[libx264 @ 0x5629c9437180] frame P:378   Avg QP:37.95  size: 86873
[libx264 @ 0x5629c9437180] frame B:1116  Avg QP:40.78  size: 47748
[libx264 @ 0x5629c9437180] consecutive B-frames:  0.8%  0.0%  0.0% 99.2%
[libx264 @ 0x5629c9437180] mb I  I16..4:  3.9% 81.9% 14.2%
[libx264 @ 0x5629c9437180] mb P  I16..4:  2.5% 19.5%  0.8%  P16..4: 30.2%  6.1%  2.8%  0.0%  0.0%    skip:38.2%
[libx264 @ 0x5629c9437180] mb B  I16..4:  0.3%  3.0%  0.0%  B16..8: 33.1%  4.2%  0.4%  direct: 2.0%  skip:57.0%  L0:49.3% L1:49.2% BI: 1.5%
[libx264 @ 0x5629c9437180] 8x8 transform intra:86.8% inter:87.4%
[libx264 @ 0x5629c9437180] coded y,uvDC,uvAC intra: 52.4% 12.7% 1.2% inter: 13.5% 2.0% 0.0%
[libx264 @ 0x5629c9437180] i16 v,h,dc,p: 33% 15%  6% 46

frame= 1500 fps= 19 q=-1.0 Lsize=   23176kB time=00:00:29.94 bitrate=6341.2kbits/s speed=0.384x    
video:23157kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.080733%
[libx264 @ 0x55c5943cb180] frame I:6     Avg QP:41.98  size:130875
[libx264 @ 0x55c5943cb180] frame P:378   Avg QP:46.66  size: 23399
[libx264 @ 0x55c5943cb180] frame B:1116  Avg QP:48.85  size: 12618
[libx264 @ 0x55c5943cb180] consecutive B-frames:  0.8%  0.0%  0.0% 99.2%
[libx264 @ 0x55c5943cb180] mb I  I16..4: 17.1% 77.7%  5.2%
[libx264 @ 0x55c5943cb180] mb P  I16..4:  4.6%  4.0%  0.0%  P16..4: 24.0%  1.0%  0.8%  0.0%  0.0%    skip:65.5%
[libx264 @ 0x55c5943cb180] mb B  I16..4:  0.2%  0.2%  0.0%  B16..8: 24.1%  0.4%  0.0%  direct: 0.0%  skip:75.0%  L0:50.1% L1:49.7% BI: 0.2%
[libx264 @ 0x55c5943cb180] 8x8 transform intra:50.5% inter:98.9%
[libx264 @ 0x55c5943cb180] coded y,uvDC,uvAC intra: 12.9% 12.2% 0.8% inter: 3.5% 0.8% 0.0%
[libx264 @ 0x55c5943cb180] i16 v,h,dc,p: 41% 29%  9% 21%
[

## From YUV multires to H265 HEVC

In [2]:
import os
import subprocess
source = "midnight"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
quality = [15, 22, 28, 34, 40]
for current_crf in range(0,5):
    out = "test_vids/encoded/" + source + "/" + source + "HEVC_cf" + str(quality[current_crf]) + ".mp4"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libx265 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps=4.4 q=36.8 Lsize=  133219kB time=00:00:29.94 bitrate=36450.5kbits/s speed=0.0869x     
video:133196kB audio:0kB subtitle:0kB other streams:0kB global headers:2kB muxing overhead: 0.017442%
x265 [info]: frame I:      6, Avg QP:29.39  kb/s: 111125.47
x265 [info]: frame P:    367, Avg QP:30.96  kb/s: 68631.60
x265 [info]: frame B:   1127, Avg QP:36.04  kb/s: 25465.85
x265 [info]: Weighted P-Frames: Y:0.0% UV:0.0%
x265 [info]: consecutive B-frames: 0.3% 0.0% 0.0% 96.8% 2.9% 

encoded 1500 frames in 344.37s (4.36 fps), 36369.71 kb/s, Avg QP:34.77
ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass

## From YUV multires to VP9 WEBM

In [3]:
import os
import subprocess
source = "midnight"
path = "test_vids/refs/" + source + ".yuv"
res_width = 3840
res_height = 2160
quality = [15, 20, 25, 30, 35]
for current_crf in range(0,5):
    out = "test_vids/encoded/" + source + "/" + source + "VP9_cf" + str(quality[current_crf]) + ".webm"
    to_avc = f"ffmpeg -f rawvideo -video_size {res_width}x{res_height} \
    -r 50 -pixel_format yuv420p -i {path} -c:v libvpx-vp9 -crf {quality[current_crf]} {out}"
    subprocess.call(to_avc, shell=True)

ffmpeg version N-106635-g83e1a1de88 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 9 (Ubuntu 9.4.0-1ubuntu1~20.04.1)
  configuration: --prefix=/home/ubuntu/ffmpeg_build --pkg-config-flags=--static --extra-cflags=-I/home/ubuntu/ffmpeg_build/include --extra-ldflags=-L/home/ubuntu/ffmpeg_build/lib --extra-libs='-lpthread -lm' --ld=g++ --bindir=/home/ubuntu/bin --enable-gpl --enable-gnutls --enable-libaom --enable-libass --enable-libfdk-aac --enable-libfreetype --enable-libmp3lame --enable-libopus --enable-libsvtav1 --enable-libdav1d --enable-libvorbis --enable-libvpx --enable-libx264 --enable-libx265 --enable-libvmaf --enable-version3 --enable-nonfree
  libavutil      57. 24.101 / 57. 24.101
  libavcodec     59. 26.100 / 59. 26.100
  libavformat    59. 22.100 / 59. 22.100
  libavdevice    59.  6.100 / 59.  6.100
  libavfilter     8. 33.100 /  8. 33.100
  libswscale      6.  6.100 /  6.  6.100
  libswresample   4.  6.100 /  4.  6.100
  libpostproc    56.  5.100 / 56.  5.100

frame= 1500 fps=1.2 q=35.0 Lsize=  300415kB time=00:00:29.98 bitrate=82085.2kbits/s speed=0.0237x    
video:300402kB audio:0kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.004375%
